In [1]:
from NLPUtils.preprocessUtils import removePunctuation, removeWord
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pickle
import os

In [2]:
ROOT = '.\\SLTCDataset\\ohsumed_single_23\\'
TRAIN = 'training'
X_TRAIN_FILE = 'x_train.npy'
Y_TRAIN_FILE = 'y_train.npy'
TEST = 'test'
X_TEST_FILE = 'x_test.npy'
Y_TEST_FILE = 'y_test.npy'
VOC_FILE = 'voc.data'
MAELEN = 50

In [3]:
trainRoot = os.path.join(ROOT, TRAIN)

In [4]:
removeWordList = stopwords.words('english')

In [5]:
label2idx = {label : i for i, label in enumerate(os.listdir(trainRoot))}

In [6]:
xRawDatas = []
yRawDatas = []
voc = set()
for label in os.listdir(trainRoot):
    labelPath = os.path.join(trainRoot, label)
    for fileName in os.listdir(labelPath):
        filePath  = os.path.join(labelPath, fileName)
        with open(filePath, 'r', encoding='utf-8') as f:
            data = f.read()
        data = removePunctuation(data)
        data = removeWord(removeWordList =removeWordList, sentence=data.split())
        voc.update(data)
        xRawDatas.append(data)
        yRawDatas.append(label2idx[label])


In [7]:
word2idx = {word : i for i, word in enumerate(voc, start=1)}
word2idx.update({'<unk>' : 0})
voc.update('<unk>')
with open(os.path.join(ROOT, VOC_FILE), 'wb') as f:
    pickle.dump(voc, f)

In [8]:
assert len(yRawDatas) == len(xRawDatas) , "ERROR"

In [9]:
xDatas = []
for data in xRawDatas:
    xData = [word2idx[word] for word in data[:MAELEN]]
    while len(xData) < MAELEN:
        xData.append(0)
    xDatas.append(xData)

In [10]:
x_train = np.array(xDatas, dtype=np.int64)
y_train = np.array(yRawDatas, dtype=np.int64)
np.save(os.path.join(ROOT, X_TRAIN_FILE), x_train)
np.save(os.path.join(ROOT, Y_TRAIN_FILE), y_train)

In [11]:
testRoot = os.path.join(ROOT, TEST)
for label in os.listdir(testRoot):
    labelPath = os.path.join(testRoot, label)
    for fileName in os.listdir(labelPath):
        filePath  = os.path.join(labelPath, fileName)
        with open(filePath, 'r', encoding='utf-8') as f:
            data = f.read()
        data = removePunctuation(data)
        data = removeWord(removeWordList = removeWordList, sentence=data.split())
        xRawDatas.append(data)
        yRawDatas.append(label2idx[label])

In [12]:
assert len(yRawDatas) == len(xRawDatas) , "ERROR"

In [13]:
xDatas = []
for data in xRawDatas:
    xData = []
    for word in data[:MAELEN]:
        if word in voc:
            xData.append(word2idx[word])
        else:
            xData.append(0)
    while len(xData) < MAELEN:
        xData.append(0)
    xDatas.append(xData)

In [14]:
x_test = np.array(xDatas, dtype=np.int64)
y_test = np.array(yRawDatas, dtype=np.int64)
np.save(os.path.join(ROOT, X_TEST_FILE), x_test)
np.save(os.path.join(ROOT, Y_TEST_FILE), y_test)

In [15]:
print(x_test.shape)
print(y_test.shape)

(7400, 50)
(7400,)
